DRESS COLOR CLASSIFICATION
---------------------------

Connect the google drive to google colab

In [ ]:

from google.colab import drive
drive.mount('/content/drive')



Import library and import dataset from drive 

In [ ]:
import tensorflow as tf


data_dir = '/content/drive/MyDrive/Colab Notebooks/dress_dataset'

for root, dirs, files in os.walk(data_dir):
    print("Root:", root)
    print("Dirs:", dirs)
    print("Files:", files)
